In [3]:
from chess_game import ChessGame
from model.gaming_model import GamingRLModel
from chess_trainer import ChessTrainer
from engine import ChessEngine
import chess as python_chess

model = GamingRLModel("../gaming_model.keras")
chess = ChessGame()
engine = ChessEngine(model)
trainer = ChessTrainer(engine)

board = chess.get_initial_state()
board.push(python_chess.Move.from_uci("e2e4"))

Model loaded from /Users/mbandeira/workspace/chess2/src/../gaming_model.keras


In [ ]:
move,_,_ = engine.best_move(board, 25, 0)
print(move)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
(4672,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/s

In [ ]:
board = chess.get_initial_state()

# results = trainer.play_game(board, 100)
results = trainer.generate_games("../games_data", 720, 25)
print(results)

In [ ]:
trainer.save_games(results, "../games_data")

In [ ]:
results = trainer.load_games("../games_data")

In [ ]:
import numpy as np

flat_results = [step for game in results for step in game]

model.train(flat_results)